In [1]:
%run 'librairies.ipynb'

In [2]:
import requests
from collections import ChainMap

In [3]:
params = {
    'idSite': 1,
    'module': 'API',
    'method': 'API.get',
    'format': 'json',
    'period': 'day',
    'date': '2014-01-01,2019-12-31',
}

def get_stats(year):
    stats_url = 'https://stats.data.gouv.fr/index.php'
    
    params['date'] = f'{year}-01-01,{year}-12-31'
    
    r = requests.get(stats_url, params=params)
    
    print(r.url)
    print(r.status_code)
    
    return r.json()

In [4]:
years = range(2014, 2020)

In [5]:
freq_years = [ get_stats(y) for y in years ]

https://stats.data.gouv.fr/index.php?idSite=1&module=API&method=API.get&format=json&period=day&date=2014-01-01%2C2014-12-31
200
https://stats.data.gouv.fr/index.php?idSite=1&module=API&method=API.get&format=json&period=day&date=2015-01-01%2C2015-12-31
200
https://stats.data.gouv.fr/index.php?idSite=1&module=API&method=API.get&format=json&period=day&date=2016-01-01%2C2016-12-31
200
https://stats.data.gouv.fr/index.php?idSite=1&module=API&method=API.get&format=json&period=day&date=2017-01-01%2C2017-12-31
200
https://stats.data.gouv.fr/index.php?idSite=1&module=API&method=API.get&format=json&period=day&date=2018-01-01%2C2018-12-31
200
https://stats.data.gouv.fr/index.php?idSite=1&module=API&method=API.get&format=json&period=day&date=2019-01-01%2C2019-12-31
200


In [6]:
df = pd.DataFrame(dict(ChainMap(*freq_years))).transpose()

In [7]:
# df = pd.DataFrame(get_stats()).transpose()

In [8]:
df.reset_index(inplace=True)
df.sort_values('index', inplace=True)
df.head()

,index,nb_uniq_visitors,nb_visits,nb_users,nb_actions,max_actions,bounce_count,sum_visit_length,Referrers_visitorsFromSearchEngines,Referrers_visitorsFromSocialNetworks,...,nb_users_returning,max_actions_returning,nb_conversions_new_visit,nb_visits_converted_new_visit,revenue_new_visit,conversion_rate_new_visit,nb_conversions_returning_visit,nb_visits_converted_returning_visit,revenue_returning_visit,conversion_rate_returning_visit
1826,2014-01-01,750,835,0,3467,78,385,168799,162,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1827,2014-01-02,1795,2013,0,10634,184,799,515282,423,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1828,2014-01-03,1549,1701,0,8594,165,686,422992,415,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1829,2014-01-04,813,909,0,4329,537,420,253053,238,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1830,2014-01-05,912,1000,0,4152,94,479,191646,273,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.to_csv('frequentation-2014-2019.csv')

In [10]:
df.count()

index                                  2191
nb_uniq_visitors                       2191
nb_visits                              2191
nb_users                               2191
nb_actions                             2191
                                       ... 
conversion_rate_new_visit               246
nb_conversions_returning_visit          247
nb_visits_converted_returning_visit     247
revenue_returning_visit                 247
conversion_rate_returning_visit         247
Length: 65, dtype: int64

In [12]:
alt.Chart(df).mark_line().encode(
    x="month(index):T",
    y="sum(nb_visits):Q",
    color="year(index):N"
)

alt.Chart(...)

In [13]:
alt.Chart(df).mark_area().encode(
    x="index:T",
    y="cumulative_count:Q",
).transform_window(
    window=[{ "op": "sum", "field": "nb_visits", "as": "cumulative_count"}],
    sort=[{'field': "index"}]
)

alt.Chart(...)

In [14]:
alt.Chart(df).mark_line().encode(
    x="yearmonth(index):T",
    y="average(avg_time_on_site):Q",
)

alt.Chart(...)

In [15]:
alt.Chart(df).mark_line().encode(
    x="month(index):T",
    y="average(avg_time_on_site):Q",
    color="year(index):N"
)

alt.Chart(...)

In [16]:
alt.Chart(df).mark_line().encode(
    x="yearmonth(index):T",
    y="average(bounce_rate):Q",
).transform_calculate(
    bounce_rate = alt.datum.bounce_count / alt.datum.nb_visits
)

alt.Chart(...)

In [17]:
alt.Chart(df).mark_line().encode(
    x="month(index):T",
    y="average(bounce_rate):Q",
    color="year(index):N"
).transform_calculate(
    bounce_rate = alt.datum.bounce_count / alt.datum.nb_visits
)

alt.Chart(...)